<h3>Next Steps</h3>
<ul>
    <li>figure out why num_ebikes_available does not always match vehicle_type_id = 2 (sometimes greater, sometimes less)
        <li>why are there 1664 stations (should be less)
            <li>what do is_renting and is_returning mean?
</ul>

<h3>Documentation</h3>
Live Station Data from Divvy Site
https://gbfs.divvybikes.com/gbfs/2.3/gbfs.json

### Findings
837 have "public rack" in name, 828 don't

# import libraries

In [1]:
import pandas as pd
import requests

# review vehicle types

In [112]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/vehicle_types.json?"

#run the query
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [114]:
#navigate through dictionaries to find station data
#data.keys()
data['data'].keys()

dict_keys(['vehicle_types'])

In [117]:
#look at all data
data['data']['vehicle_types']

[{'vehicle_type_id': '1',
  'form_factor': 'bicycle',
  'propulsion_type': 'human'},
 {'max_range_meters': 54717.56,
  'vehicle_type_id': '2',
  'form_factor': 'bicycle',
  'propulsion_type': 'electric_assist'},
 {'max_range_meters': 70810.95999999999,
  'vehicle_type_id': '3',
  'form_factor': 'scooter',
  'propulsion_type': 'electric'}]

# import station info

In [131]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_information.json?"

#request data
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [132]:
#navigate through dictionaries to find station data
#data.keys()
data['data'].keys()

dict_keys(['stations'])

In [135]:
#look at the first station
data['data']['stations'][0]

{'lon': -87.7011951301,
 'name': 'Richmond St & Diversey Ave',
 'station_id': 'a3b00b3a-a135-11e9-9cda-0a87ae2ba916',
 'lat': 41.93190196886,
 'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
  'ios': 'https://chi.lft.to/lastmile_qr_scan'},
 'short_name': '15645',
 'capacity': 15}

In [140]:
df_station_info = pd.DataFrame(data['data']['stations'], columns=['station_id','name','lat','lon'])
df_station_info.head()

station_id                        name  \
0  a3b00b3a-a135-11e9-9cda-0a87ae2ba916  Richmond St & Diversey Ave   
1  a60781db-2bec-4a15-a4a2-363297aefcec        Halsted St & 73rd St   
2  a3aae35a-a135-11e9-9cda-0a87ae2ba916       Calumet Ave & 35th St   
3  a3b23cb8-a135-11e9-9cda-0a87ae2ba916        Racine Ave & 65th St   
4  a3a79adb-a135-11e9-9cda-0a87ae2ba916       Green St & Madison St   

         lat        lon  
0  41.931902 -87.701195  
1  41.761605 -87.644161  
2  41.831379 -87.618034  
3  41.775393 -87.654308  
4  41.881859 -87.649264

# Import station status

In [11]:
# select = "date_extract_y(date) as Year,date_extract_m(date) as Month,count(distinct date) as nDays,sum(rides) as nRides"
# where = "year between 2019 and 2022"
# group_by = "Year, Month"
# limit = 9999

# url = f"https://data.cityofchicago.org/resource/5neh-572f.json?$SELECT={select}&$WHERE={where}&$GROUP={group_by}&$LIMIT={limit}"
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_status.json?"
url

'https://gbfs.lyft.com/gbfs/2.3/chi/en/station_status.json?'

In [104]:
#run the query
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [105]:
#navigate through dictionaries to find station data
# data.keys()
data['data'].keys()

dict_keys(['stations'])

In [106]:
#look at the first station
data_stations = data['data']['stations']
data_stations[0]

{'num_bikes_available': 5,
 'num_scooters_unavailable': 2,
 'vehicle_types_available': [{'vehicle_type_id': '1', 'count': 5},
  {'vehicle_type_id': '2', 'count': 0},
  {'vehicle_type_id': '3', 'count': 0}],
 'station_id': 'a3a5700b-a135-11e9-9cda-0a87ae2ba916',
 'is_renting': 1,
 'num_bikes_disabled': 3,
 'num_ebikes_available': 0,
 'is_returning': 1,
 'num_docks_disabled': 0,
 'vehicle_docks_available': [{'count': 17,
   'vehicle_type_ids': ['1', '2', '3']}],
 'num_scooters_available': 0,
 'num_docks_available': 17,
 'is_installed': 1,
 'last_reported': 1700102268}

In [145]:
#read into dataframe and rename columns
df_stations= pd.DataFrame(data_stations, columns=
        ['station_id','num_bikes_available','vehicle_types_available','num_bikes_disabled','num_docks_available','num_ebikes_available'])
df_stations.rename(columns={
    'num_bikes_available':'nGoodBikes',
    'num_bikes_disabled':'nBrokenBikes',
    'num_docks_available':'nDocksOpen'
}, inplace=True)

#add calculated columns
df_stations['nClassic'] = df_stations['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '1'), 0))
df_stations['nElectric'] = df_stations['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '2'), 0))
# df_stations['nScooters'] = df_stations['vehicle_types_available'].apply(
#     lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '3'), 0))
df_stations['nAllBikes']=df_stations['nGoodBikes']+df_stations['nBrokenBikes']
df_stations.drop('vehicle_types_available',axis=1, inplace=True)
df_stations['hasNoBikes']=(df_stations['nGoodBikes']==0)
df_stations['hasNoDocks']=(df_stations['nDocksOpen']==0)
df_stations.head()

station_id  nGoodBikes  nBrokenBikes  nDocksOpen  \
0  a3a5700b-a135-11e9-9cda-0a87ae2ba916           5             3          17   
1                   1867616662773391950           8             1           6   
2  a3a547b8-a135-11e9-9cda-0a87ae2ba916           5             0          10   
3  a3a96aca-a135-11e9-9cda-0a87ae2ba916           8             0           1   
4  a3a41004-a135-11e9-9cda-0a87ae2ba916           6             0           9   

   num_ebikes_available  nClassic  nElectric  nAllBikes  hasNoBikes  \
0                     0         5          0          8       False   
1                     7         1          7          9       False   
2                     1         4          1          5       False   
3                     6         2          6          8       False   
4                     1         5          1          6       False   

   hasNoDocks  
0       False  
1       False  
2       False  
3       False  
4       False

# Merge Station Info

In [146]:
df_stations = pd.merge(df_stations, df_station_info, on='station_id')
df_stations

station_id  nGoodBikes  nBrokenBikes  \
0     a3a5700b-a135-11e9-9cda-0a87ae2ba916           5             3   
1                      1867616662773391950           8             1   
2     a3a547b8-a135-11e9-9cda-0a87ae2ba916           5             0   
3     a3a96aca-a135-11e9-9cda-0a87ae2ba916           8             0   
4     a3a41004-a135-11e9-9cda-0a87ae2ba916           6             0   
...                                    ...         ...           ...   
1659                   1674190656158837882           0             0   
1660                   1674190428525570800           0             0   
1661                   1674190239547009458           0             0   
1662                   1806749727197839828           0             0   
1663                   1715846705501012842           1             0   

      nDocksOpen  num_ebikes_available  nClassic  nElectric  nAllBikes  \
0             17                     0         5          0          8   
1              6                     7         1          7          9   
2             10                     1         4          1          5   
3              1                     6         2          6          8   
4              9                     1         5          1          6   
...          ...                   ...       ...        ...        ...   
1659           1                     0         0          0          0   
1660           1                     0         0          0          0   
1661           6                     0         0          0          0   
1662           4                     0         0          0          0   
1663           0                     1         0          0          1   

      hasNoBikes  hasNoDocks                                         name  \
0          False       False                        Dearborn St & Erie St   
1          False       False                    Karlov Ave & Armitage Ave   
2          False       False                                63rd St Beach   
3          False       False                         Halsted St & 37th St   
4          False       False                    Cannon Dr & Fullerton Ave   
...          ...         ...                                          ...   
1659        True       False        Public Rack - Wentworth Ave & 79th St   
1660        True       False        Public Rack - Lake Park Ave & 44th St   
1661        True       False   Public Rack - Christiana Ave & Belmont Ave   
1662        True       False     Public Rack - Milwaukee Ave & Warner Ave   
1663       False        True  Public Rack - Independence Blvd & Roosevelt   

            lat        lon  
0     41.893992 -87.629318  
1     41.917480 -87.728952  
2     41.780911 -87.576324  
3     41.827059 -87.645720  
4     41.926756 -87.634429  
...         ...        ...  
1659  41.750712 -87.629949  
1660  41.814697 -87.598006  
1661  41.939170 -87.711690  
1662  41.956259 -87.750766  
1663  41.866212 -87.718497  

[1664 rows x 13 columns]

# Export for analysis

In [147]:
df_stations.to_csv("../results/station_summaries.csv")

# Summarize Data

In [148]:
#total # of bikes
df_stations['nAllBikes'].sum()

6713

In [149]:
#calculate percent broken bikes
100*df_stations['nBrokenBikes'].sum()/df_stations['nAllBikes'].sum()

9.325189929986593

# Quality Check

In [150]:
#nElectric doesn't always match num_ebikes_available, can be less than or greater
#df_stations[df_stations['nElectric']!=df_stations['num_ebikes_available']]

In [151]:
#confirm totals
df_stations['nBikes_check']= df_stations['nClassic'] + df_stations['nElectric']
df_stations[df_stations['nBikes']!=df_stations['nBikes_check']]

KeyError: 'nBikes'